In [1]:
import math
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
from sklearn.utils import shuffle
import nltk
import gensim
from gensim.models import Word2Vec
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

2023-07-11 17:40:57.208363: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 17:40:57.332930: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-11 17:40:57.795900: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-11 17:40:57.795963: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

### Building the Vocabulary

First, we build the vocabulary dictionaries for the source and target. 
The vocabulary is the the file `vocab.txt` (generated in the other script).

In [2]:
# Word string -> ID mapping
dictionary = dict()

with open('data/vocab.30K.txt', encoding='utf-8') as f:
    for line in f:
        # disregard the new line aka `\n`
        dictionary[line[:-1]] = len(dictionary)
        
reverse_dictionary = dict(zip(dictionary.values(),dictionary.keys()))

print('Dictionary:', list(dictionary.items())[:10], end = '\n')
print('Reverse dictionary:', list(reverse_dictionary.items())[:10], end = '\n')
print('Vocabulary size: ', len(dictionary), end = '\n')


Dictionary: [('<unk>', 0), ('<s>', 1), ('</s>', 2), ('.', 3), ('the', 4), (',', 5), ('a', 6), ('?', 7), ('to', 8), ('you', 9)]
Reverse dictionary: [(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, '.'), (4, 'the'), (5, ','), (6, 'a'), (7, '?'), (8, 'to'), (9, 'you')]
Vocabulary size:  30000


### Loading the Dataset
Here we load the data from the `dataset.csv` file (generated in the other script)

In [3]:
dataset = pd.read_csv('data/dataset.csv')

### Data pre-processing
Transform to lower, remove the new line and the punctuation

In [4]:
def lowerDataset(data):
    return data.str.lower() 
    
def cleanDataset(data):
    return data.str.replace('/r/','')                  \
                .str.replace(')','', regex=False)      \
                .str.replace('(','', regex=False)      \
                .str.replace(']','', regex=False)      \
                .str.replace('[','', regex=False)      \
                .str.replace('!','')                   \
                .str.replace('"','')                   \
    
def paddDataset(data):
    return data.str.replace(',', ' ,')                 \
                .str.replace('.',' . ', regex=False)    \
                .str.replace('?',' ?', regex=False)    \
                .str.replace('\n',' ')


In [5]:
wt = nltk.tokenize.WhitespaceTokenizer()
for column in dataset.columns:    
    dataset[column] = lowerDataset(dataset[column]) 
    dataset[column] = cleanDataset(dataset[column])
    dataset[column] = paddDataset(dataset[column])                                    
    dataset[column] = dataset[column].apply(wt.tokenize)
dataset = shuffle(dataset)

In [6]:
dataset.head()

,question,answer
863578,"[reddit, ,, what, are, you, listening, to, rig...","[last, hope, -, paramore, ., some, good, soft,..."
472061,"[serious, what, do, you, regret, the, most, in...","[doing, heroin, for, the, first, time, ., i, w..."
17364,"[what, is, the, worst, song, to, play, at, a, ...","[hooker, with, a, penis, -, tool]"
734580,"[night, shift, workers, of, reddit, ,, what, w...","[i, worked, the, first, shift, at, a, burger, ..."
984743,"[theme, park, employees, ,, what, are, some, c...","[so, i, work, at, california's, great, america..."


### Data analysis
Mean sentence length and standard deviation of sentence length

In [7]:
print('Central tendency, dispersion and shape of questions’s distribution')
print(dataset['question'].str.len().describe().apply(lambda x: format(x, 'f')))

Central tendency, dispersion and shape of questions’s distribution
count    1149819.000000
mean          17.113906
std            9.139078
min            1.000000
25%           11.000000
50%           15.000000
75%           21.000000
max           82.000000
Name: question, dtype: object


In [8]:
print('Central tendency, dispersion and shape of answers’s distribution')
print(dataset['answer'].str.len().describe().apply(lambda x: format(x, 'f')))

Central tendency, dispersion and shape of answers’s distribution
count    1149819.000000
mean          54.452828
std          844.371854
min            0.000000
25%           10.000000
50%           22.000000
75%           53.000000
max       563680.000000
Name: answer, dtype: object


### Update the sentences to fixed length
Update all sentences with a fixed size, to process the sentences as batches.

In [7]:
max_sent_length = {'question' : 30, 'answer': 60}

def padding_sent(source):
    padded = []
    for tokens in dataset[source]: 
        # adding the start token
        tokens.insert(0, '<s>')  

        if len(tokens) >= max_sent_length[source]:
            tokens = tokens[:(max_sent_length[source] - 1)]
            tokens.append('</s>')

        if len(tokens) < max_sent_length[source]:
            tokens.extend(['</s>' for _ in range(max_sent_length[source] - len(tokens))])  

        padded.append(tokens)
    return padded

In [8]:
questions = padding_sent('question')
answers = padding_sent('answer')

### Create the reverse dataset
The reverse dataset are going to be used to retrieve the decoder output to text

In [9]:
def create_reverse_dataset(source):
    reverse_tokens = []
    reverse_dataset = []
    for tokens in source: 
        for token in tokens: 
            if token not in dictionary.keys():
                reverse_tokens.append(dictionary['<unk>'])
            else:
                reverse_tokens.append(dictionary[token])
        reverse_dataset.append(reverse_tokens)
        reverse_tokens = []
    return reverse_dataset

inputs_indexes =  np.array(create_reverse_dataset(questions), dtype=np.int32)
outputs_indexes =  np.array(create_reverse_dataset(answers), dtype=np.int32)

### Word Embedding
Use the Word2Vec to embed the input to a hight demention vector, that will keep the word relationships 

In [11]:
"""
Learns all the Word2Vec relationships fo the questions, answers and unkown words
This code needs to be run just once
"""

model = Word2Vec(questions + answers + [['<unk>']], vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [10]:
model = Word2Vec.load("word2vec.model")

### Formating the inputs and outputs

The dataset original format for inputs is 30x100 and for the outputs is 60x30000 for 1.149 million records, making the memory usage impracticable.

The inputs and outputs are goint to be refactor to 15x100 for the inputs and 30x30000 for the outputs to a total of 2.299 million records.

In [11]:
"""

input_window_size = 15
output_window_size = 30

"""

#Factor to reshape the dataset
reshape_factor = 2 

input_window_size = int(max_sent_length['question'] / reshape_factor)
output_window_size = int(max_sent_length['answer'] / reshape_factor)

def array_numpy(array):
    return np.array(array, dtype=np.float32)

def get_batch_inputs(batch, batch_size = 10):
    train_inputs = list()
    
    for input_index in inputs_indexes[batch:batch + batch_size]:
        train_input = list()   
        
        for index in input_index:
            # Formates the input to the word2vec encoded format
            train_input.append(model.wv[reverse_dictionary[index]])
            
        train_inputs.append(array_numpy(train_input[:input_window_size]))
        train_inputs.append(array_numpy(train_input[input_window_size:]))
    return array_numpy(train_inputs)

def get_batch_outputs(batch, batch_size = 10):
    train_outputs = list()
    train_targets = list()
    
    for output_index in outputs_indexes[batch:batch + batch_size]:
        train_output = list()
        train_target = list()
                
        for timestep, index in enumerate(output_index):
            # Formates the output to the one-hot-encode format
            output_encoded = np.zeros(len(dictionary), dtype=np.float32)
            output_encoded[index] = 1
            train_output.append(output_encoded)
            
            # Formates the target to the one-hot-encode format
            # Setted as index - 1 because it ignores the first <s>
            if timestep > 0:
                target_encoded = np.zeros(len(dictionary), dtype=np.float32)
                target_encoded[output_index[timestep]] = 1.0
                train_target.append(target_encoded)
        
        train_outputs.append(array_numpy(train_output[:output_window_size]))
        train_outputs.append(array_numpy(train_output[output_window_size:]))
        
        #Add a </s> in the end of the target so len(output) == len(taget) 
        target_encoded = np.zeros(len(dictionary), dtype=np.float32)
        target_encoded[output_index[-1]] = 1.0
        train_target.append(target_encoded)
                
        train_targets.append(array_numpy(train_target[:output_window_size]))
        train_targets.append(array_numpy(train_target[output_window_size:]))
        
    return array_numpy(train_outputs), array_numpy(train_targets)

In [12]:
input_example = get_batch_inputs(0, 1)
output_example, target_example = get_batch_outputs(0, 1)

In [13]:
print("Input shape:", input_example.shape)
print("Output shape:", output_example.shape)
print("Target shape:", target_example.shape)

Input shape: (2, 15, 100)
Output shape: (2, 30, 30000)
Target shape: (2, 30, 30000)


In [14]:
# Input shape = 100
input_shape = input_example.shape[2]

# Output shape = 30000
output_shape = output_example.shape[2]

#Dimensionality
dimensionality = 256

#The batch size and number of epochs
batch_size = 10
epochs = 150

#Encoder
encoder_inputs = Input(shape=(None, input_shape))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

#Decoder
decoder_inputs = Input(shape=(None, output_shape))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(output_shape, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Compiling
training_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')


2023-07-11 17:43:35.446199: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-11 17:43:35.478370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-11 17:43:35.478551: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-11 17:43:35.478949: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### Training in batches

Due to memory usage, the training will be divided into batches

In [15]:
num_records = len(dataset) * reshape_factor
data_batch_size = 50
supported_batch_records = num_records / data_batch_size
batch_records = math.ceil(supported_batch_records)

In [16]:
print("Using ", batch_records, " batches")
print("With ", data_batch_size, " (inputs, outputs) each")
print("With the totall of ", num_records, " pairs to process")

Using  45993  batches
With  50  (inputs, outputs) each
With the totall of  2299638  pairs to process


### Cleaning memory

Removing variables that are no longer needed before start the training

In [17]:
del dataset
del questions
del answers

### Start the training

In [18]:
# Run for the batches 
for batch_record in range(batch_records + 1):
    
    # Validate for the limit size of the dataset
    # Limit the last step not to exceed the data size
    if batch_record > (num_records / data_batch_size):
        input_batch_size = math.floor((supported_batch_records % 1) * data_batch_size)
    
    # Get the inputs
    inputs_data = get_batch_inputs(batch_record, data_batch_size)
    
    # Get the outputs
    outputs_data, target_data = get_batch_outputs(batch_record, data_batch_size)
    
    #Training
    training_model.fit([inputs_data, outputs_data], outputs_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
    training_model.save('training_model.h5')



Epoch 1/150


2023-07-11 17:43:52.239846: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600


8/8 [==============================] - 4s 205ms/step - loss: 10.0199 - accuracy: 0.4275 - val_loss: 9.0680 - val_accuracy: 0.4533
Epoch 2/150
8/8 [==============================] - 1s 122ms/step - loss: 7.4042 - accuracy: 0.5183 - val_loss: 6.1945 - val_accuracy: 0.4533
Epoch 3/150
8/8 [==============================] - 1s 123ms/step - loss: 4.4377 - accuracy: 0.5183 - val_loss: 4.9472 - val_accuracy: 0.4533
Epoch 4/150
8/8 [==============================] - 1s 126ms/step - loss: 3.3941 - accuracy: 0.5183 - val_loss: 5.0993 - val_accuracy: 0.4533
Epoch 5/150
8/8 [==============================] - 1s 122ms/step - loss: 3.4368 - accuracy: 0.5183 - val_loss: 5.2685 - val_accuracy: 0.4533
Epoch 6/150
8/8 [==============================] - 1s 122ms/step - loss: 3.3446 - accuracy: 0.5183 - val_loss: 5.2588 - val_accuracy: 0.4533
Epoch 7/150
8/8 [==============================] - 1s 122ms/step - loss: 3.3508 - accuracy: 0.5183 - val_loss: 5.3026 - val_accuracy: 0.4533
Epoch 8/150
8/8 [=======

Epoch 59/150
8/8 [==============================] - 1s 124ms/step - loss: 2.4423 - accuracy: 0.5525 - val_loss: 5.7379 - val_accuracy: 0.4483
Epoch 60/150
8/8 [==============================] - 1s 123ms/step - loss: 2.4217 - accuracy: 0.5562 - val_loss: 5.6650 - val_accuracy: 0.4300
Epoch 61/150
8/8 [==============================] - 1s 123ms/step - loss: 2.4016 - accuracy: 0.5533 - val_loss: 5.6714 - val_accuracy: 0.4333
Epoch 62/150
8/8 [==============================] - 1s 122ms/step - loss: 2.3783 - accuracy: 0.5550 - val_loss: 5.7403 - val_accuracy: 0.4317
Epoch 63/150
8/8 [==============================] - 1s 126ms/step - loss: 2.3644 - accuracy: 0.5583 - val_loss: 5.7189 - val_accuracy: 0.4350
Epoch 64/150
8/8 [==============================] - 1s 123ms/step - loss: 2.3430 - accuracy: 0.5554 - val_loss: 5.7232 - val_accuracy: 0.4300
Epoch 65/150
8/8 [==============================] - 1s 124ms/step - loss: 2.3268 - accuracy: 0.5583 - val_loss: 5.7702 - val_accuracy: 0.4283
Epoch 

8/8 [==============================] - 1s 122ms/step - loss: 1.2676 - accuracy: 0.7571 - val_loss: 5.7522 - val_accuracy: 0.4783
Epoch 117/150
8/8 [==============================] - 1s 123ms/step - loss: 1.2422 - accuracy: 0.7688 - val_loss: 5.7197 - val_accuracy: 0.4817
Epoch 118/150
8/8 [==============================] - 1s 123ms/step - loss: 1.2181 - accuracy: 0.7754 - val_loss: 5.7889 - val_accuracy: 0.4767
Epoch 119/150
8/8 [==============================] - 1s 122ms/step - loss: 1.1952 - accuracy: 0.7808 - val_loss: 5.7796 - val_accuracy: 0.4817
Epoch 120/150
8/8 [==============================] - 1s 123ms/step - loss: 1.1711 - accuracy: 0.7912 - val_loss: 5.7822 - val_accuracy: 0.4833
Epoch 121/150
8/8 [==============================] - 1s 125ms/step - loss: 1.1496 - accuracy: 0.7950 - val_loss: 5.7151 - val_accuracy: 0.4900
Epoch 122/150
8/8 [==============================] - 1s 123ms/step - loss: 1.1285 - accuracy: 0.8075 - val_loss: 5.7871 - val_accuracy: 0.4850
Epoch 123/150

...

8/8 [==============================] - 1s 124ms/step - loss: 0.0281 - accuracy: 1.0000 - val_loss: 4.6685 - val_accuracy: 0.6217
Epoch 139/150
8/8 [==============================] - 1s 124ms/step - loss: 0.0277 - accuracy: 1.0000 - val_loss: 4.6856 - val_accuracy: 0.6200
Epoch 140/150
8/8 [==============================] - 1s 123ms/step - loss: 0.0272 - accuracy: 1.0000 - val_loss: 4.7099 - val_accuracy: 0.6183
Epoch 141/150
8/8 [==============================] - 1s 124ms/step - loss: 0.0268 - accuracy: 1.0000 - val_loss: 4.7003 - val_accuracy: 0.6183
Epoch 142/150
8/8 [==============================] - 1s 124ms/step - loss: 0.0264 - accuracy: 1.0000 - val_loss: 4.6826 - val_accuracy: 0.6217
Epoch 143/150
8/8 [==============================] - 1s 123ms/step - loss: 0.0260 - accuracy: 1.0000 - val_loss: 4.6794 - val_accuracy: 0.6217
Epoch 144/150
8/8 [==============================] - 1s 122ms/step - loss: 0.0256 - accuracy: 1.0000 - val_loss: 4.6823 - val_accuracy: 0.6217
Epoch 145/150

Epoch 46/150
8/8 [==============================] - 1s 123ms/step - loss: 0.0124 - accuracy: 1.0000 - val_loss: 5.7691 - val_accuracy: 0.5550
Epoch 47/150
8/8 [==============================] - 1s 123ms/step - loss: 0.0123 - accuracy: 1.0000 - val_loss: 5.7655 - val_accuracy: 0.5550
Epoch 48/150
8/8 [==============================] - 1s 123ms/step - loss: 0.0121 - accuracy: 1.0000 - val_loss: 5.7713 - val_accuracy: 0.5550
Epoch 49/150
8/8 [==============================] - 1s 122ms/step - loss: 0.0120 - accuracy: 1.0000 - val_loss: 5.7700 - val_accuracy: 0.5550
Epoch 50/150
8/8 [==============================] - 1s 124ms/step - loss: 0.0119 - accuracy: 1.0000 - val_loss: 5.7707 - val_accuracy: 0.5550
Epoch 51/150
8/8 [==============================] - 1s 124ms/step - loss: 0.0118 - accuracy: 1.0000 - val_loss: 5.7699 - val_accuracy: 0.5550
Epoch 52/150
8/8 [==============================] - 1s 123ms/step - loss: 0.0116 - accuracy: 1.0000 - val_loss: 5.7695 - val_accuracy: 0.5567
Epoch 

Epoch 104/150
8/8 [==============================] - 1s 123ms/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 5.7874 - val_accuracy: 0.5583
Epoch 105/150
8/8 [==============================] - 1s 124ms/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 5.7868 - val_accuracy: 0.5583
Epoch 106/150
8/8 [==============================] - 1s 124ms/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 5.7901 - val_accuracy: 0.5600
Epoch 107/150
8/8 [==============================] - 1s 124ms/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 5.7875 - val_accuracy: 0.5567
Epoch 108/150
8/8 [==============================] - 1s 122ms/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 5.7890 - val_accuracy: 0.5583
Epoch 109/150
8/8 [==============================] - 1s 125ms/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 5.7906 - val_accuracy: 0.5583
Epoch 110/150
8/8 [==============================] - 1s 124ms/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 5.7922 - val_accuracy: 0.5583

2023-07-11 17:51:29.213556: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 160.84MiB (rounded to 168655104)requested by op gradients/CudnnRNN_grad/CudnnRNNBackprop
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-07-11 17:51:29.213625: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-07-11 17:51:29.213663: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 41, Chunks in use: 41. 10.2KiB allocated for chunks. 10.2KiB in use in bin. 208B client-requested in use in bin.
2023-07-11 17:51:29.213690: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-07-11 17:5

InternalError: Graph execution error:

Failed to call ThenRnnBackward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 30000, 256, 1, 30, 10, 256] 
	 [[{{node gradients/CudnnRNN_grad/CudnnRNNBackprop}}]]
	 [[Adam/gradients/PartitionedCall]] [Op:__inference_train_function_4700]